In [6]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import sqlite3 as db
import datetime as dt
from sklearn.metrics import mean_squared_error,r2_score

In [7]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [8]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [9]:
#baseline request from the application layer
#outputs a list of birds at the stop and a classification based solely off the number of observations
def birdList_request(StopName: str,cnx):
    try:
        query = f'SELECT speciesCode,count(subId) as "checklists",(SELECT count(subId) FROM historicObservations hxobx WHERE hxobx.speciesCode=hsob.speciesCode) as "sightings" FROM historicObservations hsob LEFT JOIN closestStop on hsob.locId=closestStop.locId WHERE StopName = "{StopName}" GROUP BY speciesCode;'
        sightings = pd.read_sql(sql=query,con=cnx)
    #rareness at the stop
        sightings['stopGroup'] = int
        bucket = sightings['checklists'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['checklists'] <= bucket[0.15],'stopGroup'] = 1  #mythic
        sightings.loc[(sightings['checklists'] > bucket[0.15]) & (sightings['checklists'] <= bucket[0.5]),'stopGroup'] = 2  #rare
        sightings.loc[(sightings['checklists'] > bucket[0.5]) & (sightings['checklists'] < bucket[0.85]),'stopGroup'] = 3   #uncommon
        sightings.loc[(sightings['checklists'] >= bucket[0.85]) & (sightings['checklists'] <=bucket[1]),'stopGroup'] = 4    #common
    #overall rareness
        sightings['overall'] = int
        bucket = sightings['sightings'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['sightings'] <= bucket[0.15],'overall'] = 1
        sightings.loc[(sightings['sightings'] > bucket[0.15]) & (sightings['sightings'] <= bucket[0.5]),'overall'] = 2
        sightings.loc[(sightings['sightings'] > bucket[0.5]) & (sightings['sightings'] < bucket[0.85]),'overall'] = 3
        sightings.loc[(sightings['sightings'] >= bucket[0.85]) & (sightings['sightings'] <=bucket[1]),'overall'] = 4
        sightings['StopName'] = StopName
    #raise an exception if the stopName given is not valid and return a list of valid stop names
    except Exception as ex:
        raise ex
    return sightings

LSTM forecasting of common birds. Why use LSTM instead of a traditional auto-regression?
- Bird volumes at a time step are not neccessarily a function of volumes at the previous time step. It is not reasonable to assume that bird behavior (presence at a location) is explained by bird behavior in a historic time step.
- Comparatively less resource intensive. LSTM models can provide accurate forecasting without stepped re-training

In [10]:
def build_dailyDataset(speciesCode: str,StopName: str,cnx = connectDB()):
    try:
        query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode = "{speciesCode}" AND StopName = "{StopName}";'
        obsData = pd.read_sql(query,con=cnx,parse_dates=['obsDt'])
        obsData.set_index('obsDt',inplace=True)
    #resample to days
        dailyData = obsData.resample('d')
        dailyData = dailyData.mean()
    except Exception as ex:
        raise ex
    return dailyData

In [11]:
##TODO build realtime prediction function using LSTM model

In [12]:
def build_interpolatedDataset(dailyData:pd.DataFrame):
    try:
    #mask in null presence
        dailyData['mask'] = dailyData['howMany'].interpolate(method='zero',limit_direction='both',limit=21)
        dailyData.loc[(dailyData['mask'].isna() == True),'howMany'] = 0
        test_dailyData = dailyData[dailyData['howMany'].notna()].drop(columns='mask')
        
#determine the best method of interpolation for the given data
        testResults = []
        methodDict = [{'method':'linear','order':3,'limit_direction':'both','limit':21},{'method':'slinear','order':3,'limit_direction':'both','limit':21},{'method':'quadratic','order':0,'limit_direction':'both','limit':21},{'method':'cubic','order':3,'limit_direction':'both','limit':21},{'method':'spline','order':3,'limit_direction':'both','limit':21},{'method':'spline','order':5,'limit_direction':'both','limit':21},{'method':'polynomial','order':3,'limit_direction':'both','limit':21},{'method':'polynomial','order':5,'limit_direction':'both','limit':21}]
        for v in list([0.2,0.25,0.3,0.35]):
            test_dailyData['sample'] = test_dailyData['howMany'].sample(frac=v,random_state=1)
            blob = {}
            for test in methodDict:
                testName = str(test['method'])
                test_dailyData[testName] = test_dailyData['sample']
                test_dailyData[testName] = test_dailyData['sample'].interpolate(method=test['method'],order=test['order'],limit=test['limit'],limit_direction=test['limit_direction']).fillna(test_dailyData['howMany'])
                r_sqr = r2_score(y_true=test_dailyData['howMany'],y_pred=test_dailyData[testName])
                if r_sqr < 0:
                    continue
                blob = {'method':test['method'],'order':test['order'],'limit':test['limit'],'sampleSize':v,'r_sqr':r_sqr}
                testResults.append(blob)
        if not testResults:
            testResults.append({'method':'nearest','order':0,'limit':21})
            resultFrame = pd.DataFrame.from_dict(testResults)
        else:
            resultFrame = pd.DataFrame.from_dict(testResults)
            resultFrame = resultFrame.groupby(by=['method','order','limit'])['r_sqr'].median().reset_index()
            resultFrame.sort_values(by=['r_sqr'],ascending=False,inplace=True,ignore_index=True)
    #apply the best interpolation method
        bestMethod = resultFrame.loc[0].to_dict()
        dailyData['howMany'] = dailyData['howMany'].interpolate(method=bestMethod['method'],order=bestMethod['order'],limit=bestMethod['limit'],limit_direction='both')
        dailyData.drop(columns='mask',inplace=True)
    except Exception as ex:
        raise ex
    return dailyData

In [13]:
def trim_DatasetDates(dailyData: pd.DataFrame):
    try:
        if (len(dailyData)/7).is_integer()!=True:
            i = 0
            for i in range(1,6):
                dailyDataCut = dailyData[i:]
                if (len(dailyDataCut)/7).is_integer()==True:
                    dailyDataCut
                    break
                else:
                    continue
        else:
            pass
    except Exception as ee:
        raise ee
    return dailyDataCut     

In [14]:
def split_trainTest(dailyObs):
    dailyObs = trim_DatasetDates(dailyData=dailyObs)
    train, test = dailyObs[1:-328],dailyObs[-328:-6]
    train = np.array(np.split(train,len(train)/7))
    test = np.array(np.split(test,len(test)/7))
    return train, test

In [104]:
#build forward-moving windows
def to_supervised(train,nInput,nOut = 7):
    train = train.reshape((train.shape[0]*train.shape[1],train.shape[2]))
    X, y = [],[]
    inStart = 0
    for _ in range(len(train)):
        inEnd = inStart + nInput
        outEnd = inEnd + nOut
        if outEnd <= len(train):
            xIn = train[inStart:inEnd,0]
            xIn = xIn.reshape((len(xIn),1))
            X.append(xIn)
            y.append(train[inEnd:outEnd,0])
        inStart+= 1
    return np.array(X),np.array(y)

In [97]:
from keras.layers import RepeatVector,Flatten,TimeDistributed

In [17]:
def build_basicModel(train,nInput):
    train_x,train_y = to_supervised(train,nInput)
    verbose, epochs, batch_size = 0,70,16
    nTimesteps,nFeatures,nOutputs = train_x.shape[1],train_x.shape[2],train_y.shape[1]
    model = Sequential()
    #model.add(Bidirectional(LSTM(units=200,activation='relu')))
    model.add(LSTM(units=200,activation='relu', input_shape=(nTimesteps,nFeatures)))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(nOutputs))
    model.compile(loss='mse',optimizer='adam')
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose)
    return model

In [98]:
def build_EncDecModel(train,nInput):
    try:
        train_x,train_y = to_supervised(train,nInput)
        verbose,epochs,batch_size = 0,20,16
        nTimesteps,nFeatures,nOutputs = train_x.shape[1],train_x.shape[2],train_y.shape[1]
    #output reshape
        train_y = train_y.reshape((train_y.shape[0],train_y.shape[1],1))
        model = Sequential()
        model.add(LSTM(200,activation='relu',input_shape=(nTimesteps,nFeatures)))
        model.add(RepeatVector(nOutputs))
        model.add(LSTM(200,activation='relu',return_sequences=True))
        model.add(TimeDistributed(Dense(100,activation='relu')))
        model.add(TimeDistributed(Dense(1)))
        model.compile(loss='mse',optimizer='adam')
        model.fit(train_x,train_y,batch_size=batch_size,verbose=verbose,epochs=epochs)
    except Exception as ex:
        raise ex
    return model

In [18]:
##TODO: build another model using encoder-decoder, run tests
##TODO export models to db

In [19]:
def forecast(model,history,nInput):
    histData = np.array(history)
    #histData = np.array(np.split(histData,len(histData)/7))
    histData = histData.reshape((histData.shape[0]*histData.shape[1],histData.shape[2]))
    inputX = histData[-nInput:,0]
    inputX = inputX.reshape((1,len(inputX),1))
    yhat = model.predict(inputX,verbose=0)
    yhat = yhat[0]
    return yhat

In [36]:
def forecast_fromDF(model,history,nInput):
    histData = np.array(history)
    histData = np.array(np.split(histData,len(histData)/7))
    histData = histData.reshape((histData.shape[0]*histData.shape[1],histData.shape[2]))
    inputX = histData[-nInput:,0]
    inputX = inputX.reshape((1,len(inputX),1))
    yhat = model.predict(inputX,verbose=0)
    yhat = yhat[0]
    return yhat

In [20]:
def eval_forecasts(actual,predicted):
    try:
        scores = list()
        for i in range(actual.shape[1]):
            mse = mean_squared_error(actual[:,i],predicted[:,i])
            rmse = np.sqrt(mse)
            scores.append(rmse)
        s = 0
        for row in range(actual.shape[0]):
            for col in range(actual.shape[1]):
                s+= (actual[row,col] - predicted[row,col])**2
        score = np.sqrt(s/(actual.shape[0]*actual.shape[1]))
    except Exception as metricExc:
        raise metricExc
    return score, scores

In [21]:
def evalModel(train,test,nInput):
    model = build_basicModel(train=train,nInput=nInput)
    history = [x for x in train]
    predictions = []
    for i in range(len(test)):
        yhatSeq = forecast(model=model,history=history,nInput=nInput)
        predictions.append(yhatSeq)
        history.append(test[i,:])
    predictions = np.array(predictions)
    score, scores = eval_forecasts(actual=test[:,:,0],predicted=predictions)
    return score, scores
    #return predictions

In [77]:
def eval_givenModel(train,test,model,nInput):
    model = model
    history = [x for x in train]
    predictions = []
    for i in range(len(test)):
        yhatSeq = forecast(model=model,history=history,nInput=nInput)
        predictions.append(yhatSeq)
        history.append(test[i,:])
    predictions = np.array(predictions)
    score, scores = eval_forecasts(actual=test[:,:,0],predicted=predictions)
    return score, scores

In [22]:
def summarizeEvalScore(name,score,scores):
    sScores = ', '.join(['%f' % s for s in scores])
    print('%s: [%.3f] %s' % (name, score, sScores))

Evaluate, test, forecast

In [24]:
commonCore = birdList_request(StopName='MargaretsWay',cnx=connectDB())
commonCore = commonCore[commonCore.apply(lambda x: (x['stopGroup']==4) and (x['overall']==4),axis=1)]
commonCore

,speciesCode,checklists,sightings,stopGroup,overall,StopName
0,amecro,39,293,4,4,MargaretsWay
3,amerob,66,371,4,4,MargaretsWay
10,bkcchi,51,281,4,4,MargaretsWay
23,chbchi,49,237,4,4,MargaretsWay
27,daejun,88,419,4,4,MargaretsWay
34,gockin,39,179,4,4,MargaretsWay
48,norfli,31,201,4,4,MargaretsWay
52,pacwre1,56,203,4,4,MargaretsWay
68,sonspa,48,323,4,4,MargaretsWay
69,spotow,50,285,4,4,MargaretsWay


In [49]:
spotow_MargWay = build_dailyDataset(speciesCode='spotow',StopName='MargaretsWay')
spotow_MargWay = build_interpolatedDataset(dailyData=spotow_MargWay)
spotow_MargWay

,howMany
obsDt,
2019-03-08,2.000000
2019-03-09,1.941176
2019-03-10,1.882353
2019-03-11,1.823529
2019-03-12,1.764706
...,...
2022-04-06,3.000000
2022-04-07,2.500000
2022-04-08,2.000000


In [74]:
train, test = split_trainTest(dailyObs=spotow_MargWay)


In [125]:
Model_spotow_MargWay = build_basicModel(train=train,nInput=7)

In [149]:
encDec_spotow_margWay = build_EncDecModel(train=train,nInput=7)

In [120]:
encDec_14 = build_EncDecModel(train=train,nInput=14)

In [130]:
encDec_21 = build_EncDecModel(train=train,nInput=21)

In [150]:
score, scores = eval_givenModel(train=train,test=test,model=encDec_spotow_margWay,nInput=7)
summarizeEvalScore(name='run',score=score,scores=scores)

run: [0.694] 0.515053, 0.491792, 0.575834, 0.594757, 1.018386, 0.798578, 0.713137


In [154]:
forecast_fromDF(model=encDec_spotow_margWay,history=spotow_MargWay[3:-7],nInput=7)

array([[3.556541 ],
       [4.145211 ],
       [4.4521904],
       [4.6746397],
       [4.895988 ],
       [5.0755334],
       [5.261075 ]], dtype=float32)

In [151]:
forecast_fromDF(model=Model_spotow_MargWay,history=spotow_MargWay[3:-7],nInput=7)

array([3.856167 , 4.0782514, 4.2023373, 4.3308506, 4.401776 , 4.492939 ,
       4.5264063], dtype=float32)

In [156]:
hist = [x for x in spotow_MargWay[3:-7].values]
hist

[array([1.82352941]),
 array([1.76470588]),
 array([1.70588235]),
 array([1.64705882]),
 array([1.58823529]),
 array([1.52941176]),
 array([1.47058824]),
 array([1.41176471]),
 array([1.35294118]),
 array([1.29411765]),
 array([1.23529412]),
 array([1.17647059]),
 array([1.11764706]),
 array([1.05882353]),
 array([1.]),
 array([1.5]),
 array([2.]),
 array([2.5]),
 array([3.]),
 array([3.5]),
 array([4.]),
 array([4.5]),
 array([5.]),
 array([5.5]),
 array([6.]),
 array([5.88571429]),
 array([5.77142857]),
 array([5.65714286]),
 array([5.54285714]),
 array([5.42857143]),
 array([5.31428571]),
 array([5.2]),
 array([5.08571429]),
 array([4.97142857]),
 array([4.85714286]),
 array([4.74285714]),
 array([4.62857143]),
 array([4.51428571]),
 array([4.4]),
 array([4.28571429]),
 array([4.17142857]),
 array([4.05714286]),
 array([3.94285714]),
 array([3.82857143]),
 array([3.71428571]),
 array([3.6]),
 array([3.48571429]),
 array([3.37142857]),
 array([3.25714286]),
 array([3.14285714]),
 arr

amerob:
1. run1: [2.551] 2.458957, 2.472114, 2.475770, 2.677192, 3.045214, 2.251491, 2.402137 n = 7
2. run2: [2.655] 2.482476, 2.564362, 2.557338, 2.738337, 3.200391, 2.457562, 2.509320 n = 21
3. run3: [3.221] 2.561701, 2.617324, 2.715710, 3.185073, 3.910806, 3.548933, 3.717976 n = 14
4. run4: [2.757] 2.647477, 2.664606, 2.711631, 2.933634, 3.188165, 2.496027, 2.599233 n = 7 bidirectional
5. run5: [2.216] 0.385453, 0.939606, 1.419237, 1.818103, 2.423542, 2.977693, 3.646165 n = 7 note: changed interpolation method to slinear
6. run6: [2.442] 0.468001, 1.068273, 1.803750, 2.283598, 2.770545, 3.216125, 3.727616 n = 7
7. run7: [2.146] 0.346224, 1.057628, 1.474756, 1.910932, 2.273262, 2.820911, 3.470625 n = 7
8. run: [2.082] 0.474444, 0.973971, 1.394103, 1.790021, 2.196929, 2.821583, 3.353662 n = 7 improved interpolation method
9. run: [2.770] 1.373590, 0.953020, 1.333994, 2.028866, 2.949500, 3.854387, 4.632758 n = 28 stick with a lower n

stejay:
1. run: [0.234] 0.171749, 0.185661, 0.209144, 0.226055, 0.248578, 0.272514, 0.297879 n = 7 for stejay

daejun:
1. run: [1.052] 1.003830, 0.517272, 0.763046, 0.948381, 1.110182, 1.257345, 1.473364 n = 7

spotow @ Margaret's Way:
1. run: [0.702] 0.232709, 0.339712, 0.556325, 0.644979, 1.098925, 0.853708, 0.788657 n = 7 on basic model
2. run: [0.876] 0.730321, 0.677835, 0.770073, 0.835399, 1.157776, 0.971914, 0.897609 n = 7 on encode-decode model
3. run: [0.788] 0.544925, 0.537754, 0.656841, 0.726218, 1.098290, 0.921656, 0.865887 n = 14 on encode-decode model
4. run: [0.648] 0.435483, 0.284990, 0.425358, 0.498536, 0.992761, 0.796883, 0.783939 n = 7 on basic model
5. run: [0.755] 0.540210, 0.536792, 0.643041, 0.715725, 1.022832, 0.858091, 0.836813 n = 21 on encode-decode model
6. run: [0.694] 0.515053, 0.491792, 0.575834, 0.594757, 1.018386, 0.798578, 0.713137 n = 7 on encode-decode model

In [ ]:
##TODO #94 build three-week forecasting method on top of LSTM model